In [2]:
# Import of relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn import preprocessing
import sklearn.metrics as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn import tree
import pickle

from sklearn.linear_model import LogisticRegression
import featureEngineering
import dataCleaning

RSEED = 42

In [19]:
df_backUp = pd.read_csv("./data/df_backUp.csv")
df_backUp.head()

,backers_count,blurb,converted_pledged_amount,country,created_at,creator,currency,currency_trailing_code,deadline,disable_communication,...,source_url,staff_pick,state,state_changed_at,static_usd_rate,usd_pledged,usd_type,category_name,category_slug,cgrouped
0,6,For my Eagle Project I am building a Prayer La...,135,US,1412399924,"{""id"":2134344211,""name"":""Inocencio Orta"",""is_r...",USD,True,1415112165,False,...,https://www.kickstarter.com/discover/categorie...,False,failed,1415112165,1.000000,135.000000,domestic,Civic Design,design/civic design,US
1,113,A first solo exhibition and book to celebrate ...,13446,GB,1415012169,"{""id"":1949162220,""name"":""Jane Moore"",""is_regis...",GBP,False,1418323578,False,...,https://www.kickstarter.com/discover/categorie...,False,successful,1418323578,1.589919,13603.343884,domestic,Illustration,art/illustration,GB
2,119,Turn loads on and off with your Arduino! Use ...,4245,US,1394997815,"{""id"":323660810,""name"":""zach Houser"",""is_regis...",USD,True,1398639684,False,...,https://www.kickstarter.com/discover/categorie...,False,successful,1398639684,1.000000,4245.000000,international,Hardware,technology/hardware,US
3,192,I am creating my first full-length album of or...,15360,US,1547337232,"{""id"":1508200079,""name"":""Alicia Viani"",""slug"":...",USD,True,1551282896,False,...,https://www.kickstarter.com/discover/categorie...,False,successful,1551282896,1.000000,15360.000000,international,Music,music,US
4,18,Photo exhibition. An environmental collaborati...,2483,NZ,1507191554,"{""id"":969290475,""name"":""Jacki Key"",""is_registe...",NZD,True,1509080400,False,...,https://www.kickstarter.com/discover/categorie...,False,failed,1509080400,0.715387,2596.856298,domestic,Fine Art,photography/fine art,Other


In [ ]:
#mBackerStaff = pickle._load(open("./models/modelBackerStaheadF.sav",'rb'))

#mStaff= pickle._load(open("./models/modelStaffRF.sav",'rb'))

## Backer

In [38]:
#prep data according to settings
df_backUpBacker = featureEngineering.prepDataFrameForPreprocessor(df_backUp.copy(),
                                                                  drop_backers_count = False, 
                                                                  drop_staff_pick = True)
print(df_backUpBacker.shape)
df_backUpBacker.head(4)

(16877, 7)


,backers_count,goal,state,category_name,cgrouped,campaign_length,time_until_launch
0,6,4500.0,0,Civic Design,US,31.39,13.35
1,113,7000.0,1,Illustration,GB,38.33,8.33
2,119,1000.0,1,Hardware,US,42.15,12.15
3,192,15000.0,1,Music,US,45.67,15.67


In [41]:
#load model
mBacker = pickle._load(open("./models/modelBackerRF.sav",'rb'))
#load preprocessor
preprocessorBacker = pickle._load(open("./models/preprocessorBackerRF.sav",'rb'))
#NON preprocess Data
y_test_Backer = df_backUpBacker["state"]
X_test_Backer = df_backUpBacker.drop("state", axis=1)
X_test_Backer = preprocessorBacker.transform(X_test_Backer)

In [42]:
y_pred_test_Backer = mBacker.predict(X_test_Backer)
print(sm.classification_report(y_test_Backer,y_pred_test_Backer))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      7413
           1       0.95      0.96      0.96      9464

    accuracy                           0.95     16877
   macro avg       0.95      0.95      0.95     16877
weighted avg       0.95      0.95      0.95     16877



## Non

In [29]:
#prep data according to settings
df_backUpNon = featureEngineering.prepDataFrameForPreprocessor(df_backUp.copy(),
                                                               drop_backers_count = True, 
                                                               drop_staff_pick = True)
print(df_backUpNon.shape)
df_backUpNon.head(4)

(16877, 6)


,goal,state,category_name,cgrouped,campaign_length,time_until_launch
0,4500.0,0,Civic Design,US,31.39,13.35
1,7000.0,1,Illustration,GB,38.33,8.33
2,1000.0,1,Hardware,US,42.15,12.15
3,15000.0,1,Music,US,45.67,15.67


In [30]:
#load model
mNon= pickle._load(open("./models/modelNoStaffNoBackersAB.sav",'rb'))
#load preprocessor
preprocessorNon = pickle._load(open("./models/preprocessorNoStaffNoBackersAB.sav",'rb'))
#NON preprocess Data
y_test_Non = df_backUpNon["state"]
X_test_Non = df_backUpNon.drop("state", axis=1)
X_test_Non = preprocessorNon.transform(X_test_Non)

In [31]:
y_pred_test_Non = mNon.predict(X_test_Non)
print(sm.classification_report(y_test_Non,y_pred_test_Non))

              precision    recall  f1-score   support

           0       0.71      0.78      0.74      7413
           1       0.81      0.75      0.78      9464

    accuracy                           0.76     16877
   macro avg       0.76      0.77      0.76     16877
weighted avg       0.77      0.76      0.76     16877



## Staff

In [32]:
#prep data according to settings
df_backUpStaff = featureEngineering.prepDataFrameForPreprocessor(df_backUp.copy(),
                                                               drop_backers_count = True, 
                                                               drop_staff_pick = False)
print(df_backUpStaff.shape)
df_backUpStaff.head(4)

(16877, 7)


,goal,staff_pick,state,category_name,cgrouped,campaign_length,time_until_launch
0,4500.0,False,0,Civic Design,US,31.39,13.35
1,7000.0,False,1,Illustration,GB,38.33,8.33
2,1000.0,False,1,Hardware,US,42.15,12.15
3,15000.0,False,1,Music,US,45.67,15.67


In [33]:
#load model
mStaff= pickle._load(open("./models/modelStaffRF.sav",'rb'))
#load preprocessor
preprocessorStaff = pickle._load(open("./models/preprocessorStaffRF.sav",'rb'))
#Staff preprocess Data
y_test_Staff = df_backUpStaff["state"]
X_test_Staff = df_backUpStaff.drop("state", axis=1)
X_test_Staff = preprocessorStaff.transform(X_test_Staff)

In [34]:
y_pred_test_Staff = mStaff.predict(X_test_Staff)
print(sm.classification_report(y_test_Staff,y_pred_test_Staff))

              precision    recall  f1-score   support

           0       0.76      0.80      0.78      7413
           1       0.83      0.80      0.82      9464

    accuracy                           0.80     16877
   macro avg       0.80      0.80      0.80     16877
weighted avg       0.80      0.80      0.80     16877



## Backer Staff

In [35]:
#prep data according to settings
df_backUpBackerStaff = featureEngineering.prepDataFrameForPreprocessor(df_backUp.copy(),
                                                               drop_backers_count = False, 
                                                               drop_staff_pick = False)
print(df_backUpBackerStaff.shape)
df_backUpBackerStaff.head(4)

(16877, 8)


,backers_count,goal,staff_pick,state,category_name,cgrouped,campaign_length,time_until_launch
0,6,4500.0,False,0,Civic Design,US,31.39,13.35
1,113,7000.0,False,1,Illustration,GB,38.33,8.33
2,119,1000.0,False,1,Hardware,US,42.15,12.15
3,192,15000.0,False,1,Music,US,45.67,15.67


In [36]:
#load model
mBackerStaff= pickle._load(open("./models/modelBackerStaffRF.sav",'rb'))
#load preprocessor
preprocessorBackerStaff = pickle._load(open("./models/preprocessorBackerStaffRF.sav",'rb'))
#BackerStaff preprocess Data
y_test_BackerStaff = df_backUpBackerStaff["state"]
X_test_BackerStaff = df_backUpBackerStaff.drop("state", axis=1)
X_test_BackerStaff = preprocessorBackerStaff.transform(X_test_BackerStaff)

In [37]:
y_pred_test_BackerStaff = mBackerStaff.predict(X_test_BackerStaff)
print(sm.classification_report(y_test_BackerStaff,y_pred_test_BackerStaff))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94      7413
           1       0.95      0.96      0.95      9464

    accuracy                           0.95     16877
   macro avg       0.95      0.95      0.95     16877
weighted avg       0.95      0.95      0.95     16877

